In [9]:
#Qmak_Calderdale_FDM_FULL_V1.0.ipynb
##last modified 20230309 

#Builds the Calderdael SUS FDM
#Assumes that the targetdb below exists
#Needs code to build it if it doesn't

#You will need to manually amend the target dataset for this script

library(bigrquery)
library(lubridate) 

# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

sourcedb <-"yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_YAS_FDM_Format"
targetdb <-'yhcr-prd-phm-bia-core.CB_FDM_YAS_V1'
targetdb <-gsub(' ','',targetdb)
print (targetdb)

[1] "yhcr-prd-phm-bia-core.CB_FDM_YAS_V1"


In [44]:
#build care sites just for Warehouse sites

#NOTE - THIS WHOLE SECTION NEEDS WORK - DO NOT RUN

sql1 <-paste('drop table if exists ',targetdb,'.care_site', sep = "")
sql9 <-paste('create table ' ,targetdb,'.care_site as select distinct a.* from yhcr-prd-phm-bia-core.CB_FDM_MASTER.care_site a 
              where a.care_site_id = 160710', sep = "")

#Now Run them 
tb1 <- bq_project_query(project_id, sql1)
tb9 <- bq_project_query(project_id, sql9)


print("Done care site")

[1] "Done care site"


In [11]:
#New section to build user table loop 

#Your loop references this table
#You run this to load the list of tables into the dataframe "table"

sql3 <-paste('select distinct table_name  from ' ,sourcedb,'.INFORMATION_SCHEMA.COLUMNS where table_name like \'tbl%\' and column_name = \'person_id\' order by table_name ;', sep = "")
tb3 <- bq_project_query(project_id, sql3)

table <- bq_table_download(tb3)
table 



eric <- paste("builder table " ,  targetdb ," done " ,Sys.time() + hours(1) ,sep = "")
print (eric)



table_name
<chr>
tbl_YAS_111
tbl_YAS_999
tbl_YAS_CAD
tbl_YAS_EPR
tbl_YAS_IUC
tbl_YAS_PTS


[1] "builder table yhcr-prd-phm-bia-core.CB_FDM_YAS_V1 done 2023-05-11 09:41:29"


In [12]:
#This then loops through the data frame until the end
# uses the two variables table[[i, 1]] and table[[i, 2]]
# in the query

sql4 <-paste('drop table if exists ',targetdb,'.tmp_persons;', sep = "")
tb4 <-bq_project_query(project_id, sql4)


sql5 <-paste('create table ',targetdb,'.tmp_persons 
(person_id int64);', sep = "")
tb5 <-bq_project_query(project_id, sql5)
#print(tb5)


#Loops through all the data tables to build a list of unique person_ids from all of them 

for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('insert into ',targetdb,'.tmp_persons select distinct cast(person_id as int64)  from ' ,sourcedb,'.',table[[i, 1]] ,';' , sep = "") 
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    temp_table <- bq_table_download(temp_queried_table)
    #print(paste0("done ", temp_table[[1, 1]]))
    #cat("\n")
}


eric <- paste("Done tmp_persons " ,  targetdb ," " ,Sys.time() + hours(1) ,sep = "")
print (eric)


[1] "Done tmp_persons yhcr-prd-phm-bia-core.CB_FDM_YAS_V1 2023-05-11 09:43:37"


In [13]:
#Now build the persons table 

sql11 <-paste('drop table if exists ' ,targetdb,'.person', sep = "")
tb11 <- bq_project_query(project_id, sql11)


sql12 <-paste('create table ' ,targetdb,'.person 
as 
SELECT distinct a.person_id
, gender_concept_id
, year_of_birth
, month_of_birth
, day_of_birth
, birth_datetime
, death_datetime
, race_concept_id
, ethnicity_concept_id
, location_id
, provider_id
, care_site_id
, person_source_value
, gender_source_value
, gender_source_concept_id
, race_source_value
, race_source_concept_id
, ethnicity_source_value
, ethnicity_source_concept_id 
FROM yhcr-prd-phm-bia-core.CB_FDM_MASTER.person a
,',targetdb,'.tmp_persons b where a.person_id =  b.person_id' , sep = "")

tb12 <- bq_project_query(project_id, sql12)
#print(sql12)


eric <- paste("person table " ,  targetdb ," built " ,Sys.time() + hours(1) ,sep = "")
print (eric)



[1] "person table yhcr-prd-phm-bia-core.CB_FDM_YAS_V1 built 2023-05-11 09:46:23"


In [ ]:
#Ensure you have updated the lookup table ie
#sql14 <- "update `yhcr-prd-phm-bia-core.CY_LOOKUPS.tbl_Dataset_ExtractDateRef` set extract_date = '2023-02-07' where DatasetName = 'CB_STAGING_DATABASE_YAS_FDM_Format'"  

#tb14 <- bq_project_query(project_id, sql14)

In [15]:


#This query makes the observation period based on the persons table BUT adds in rules for this data extract.
#This query makes the observation period based on the persons table BUT adds in rules for this data extract.

#Qmak_PrimaryCareObservationPeriod_Part1

sql15 <-paste('drop table if exists ' ,targetdb,'.tmp_Eventdates' , sep = "")
sql16 <-paste('drop table if exists ' ,targetdb,'.tmp_EventdatesValid' , sep = "")
sql17 <-paste('truncate table ' ,targetdb,'.observation_period' , sep = "")

tb15 <-bq_project_query(project_id, sql15)
tb16 <-bq_project_query(project_id, sql16)
tb17 <-bq_project_query(project_id, sql17)

#This just builds a temp table for later use 
sql18 <-paste('create table ',targetdb,'.tmp_Eventdates 
(person_id int64
, EventDate Datetime) ' , sep = "")
tb18 <-bq_project_query(project_id, sql18)


eric <- paste("temp event dates " ,  targetdb ," built " ,Sys.time() + hours(1) ,sep = "")
print (eric)


[1] "temp event dates yhcr-prd-phm-bia-core.CB_FDM_YAS_V1 built 2023-05-11 09:52:33"


In [16]:
#Loop through the user tables
#selecting person_id and date from and date to for each of them.

#start_date_first
for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('insert into ',targetdb,'.tmp_Eventdates select distinct cast(person_id as int64) as person_id, ',table[[i, 1]] ,'_start_date from ' ,sourcedb,'.',table[[i, 1]] ,' where person_id is not null;' , sep = "") 
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(temp_sql_query)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}

eric <- paste("start dates " ,  targetdb ," done " ,Sys.time() + hours(1) ,sep = "")
print (eric)


[1] "start dates yhcr-prd-phm-bia-core.CB_FDM_YAS_V1 done 2023-05-11 09:53:03"


In [17]:
#Now do end date
for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('insert into ',targetdb,'.tmp_Eventdates select distinct cast(person_id as int64) as person_id, ',table[[i, 1]] ,'_end_date from ' ,sourcedb,'.',table[[i, 1]] ,' where person_id is not null ;' , sep = "") 
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(temp_sql_query)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}
eric <- paste("end dates " ,  targetdb ," done " ,Sys.time() + hours(1) ,sep = "")
print (eric)

[1] "end dates yhcr-prd-phm-bia-core.CB_FDM_YAS_V1 done 2023-05-11 09:54:16"


In [18]:

sql24 <-paste('create table ',targetdb,'.tmp_EventdatesValid
as
select distinct a.person_id
,cast(a.EventDate as date) as EventDate
from ',targetdb,'.tmp_Eventdates a ,' 
,targetdb,'.person e 
where e.person_id = a.person_id 
and e.death_datetime is not null 
and a.EventDate >= e.birth_datetime
and a.EventDate <= date_add(e.death_datetime, INTERVAL 42 day) 
and a.EventDate <= (Select extract_date from yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_Dataset_ExtractDateRef where DatasetName = \'CB_STAGING_DATABASE_YAS_FDM_Format\')', sep = "")


#-- BUT this only includes persons with a death datetime
#-- So section below includes those with a null deathdatetime

sql25 <-paste('insert into ',targetdb,'.tmp_EventdatesValid
select distinct a.person_id
, cast(a.EventDate as date) as EventDate
from ',targetdb,'.tmp_Eventdates a
,' ,targetdb,'.person e 
where e.person_id = a.person_id 
and e.death_datetime is null 
and a.EventDate >= e.birth_datetime
and a.EventDate <= (Select extract_date from yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_Dataset_ExtractDateRef where DatasetName = \'CB_STAGING_DATABASE_YAS_FDM_Format\')', sep = "")

eric <- paste("Temp valid dates " ,  targetdb ," built " ,Sys.time() + hours(1) ,sep = "")
print (eric)



#third part pushes these into the observation_period table

sql26 <-paste('insert into ' ,targetdb,'.observation_period
select distinct
ROW_NUMBER() over (Order by person_id) as observation_period_id 
, person_id
, min(EventDate) as observation_period_start_date
, max(EventDate) as observation_period_end_date
,null as period_type_concept_id
from ',targetdb,'.tmp_EventdatesValid
group by person_id', sep = "")

#Finally Run these sql's


tb24 <-bq_project_query(project_id, sql24)
tb25 <-bq_project_query(project_id, sql25)
tb26 <-bq_project_query(project_id, sql26)

eric <- paste("observation period " ,  targetdb ," built " ,Sys.time() + hours(1) ,sep = "")
print (eric)

# Now remove from person table where there is no observation period
# This happens where there is patient data but no actual records 
# or there is patient data but no dob 

sql26a <-paste(' delete from ' ,targetdb,'.person where person_id
not in (select person_id from ', targetdb, ' . observation_period)', sep = "")

tb26a <-bq_project_query(project_id, sql26a) 

eric <- paste("invalid people " ,  targetdb ," removed " ,Sys.time() + hours(1) ,sep = "")
print (eric)



[1] "Temp valid dates yhcr-prd-phm-bia-core.CB_FDM_YAS_V1 built 2023-05-11 09:56:44"
[1] "observation period yhcr-prd-phm-bia-core.CB_FDM_YAS_V1 built 2023-05-11 09:57:04"
[1] "invalid people yhcr-prd-phm-bia-core.CB_FDM_YAS_V1 removed 2023-05-11 09:57:09"


In [19]:
# As the tables are all fdm ready we can just copy them into the FDM space
# once we've dropped them - if they exist ! 
# So drop loop next 

#start drop them first
for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('drop table if exists ',targetdb,'.', table[[i, 1]] ,';' , sep = "") 
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(temp_sql_query)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}

eric <- paste("user tables  " ,  targetdb ," removed " ,Sys.time() + hours(1) ,sep = "")
print (eric)


[1] "user tables  yhcr-prd-phm-bia-core.CB_FDM_YAS_V1 removed 2023-05-11 09:57:33"


In [21]:
#Now build them 
for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('create table ',targetdb,'.',table[[i, 1]] ,' as select src.* except(digest) from ' ,sourcedb,'.',table[[i, 1]],' src 
    , ',targetdb,'.observation_period obs where cast(src.person_id as int64)  = obs.person_id 
    and src.',table[[i, 1]],'_end_date <= obs.observation_period_end_date  
    and src.',table[[i, 1]],'_start_date >= obs.observation_period_start_date', sep = "")
    #print(temp_sql_query)
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}

eric <- paste("user tables  " ,  targetdb ," built " ,Sys.time() + hours(1) ,sep = "")
print (eric)

#But this only works for those tables with a tbl_prefix containing person_id 
# so we need to manually mop up the ones that don't.

[1] "user tables  yhcr-prd-phm-bia-core.CB_FDM_YAS_V1 built 2023-05-11 10:29:34"


In [22]:
#Section to build non tbl_ user tables 
#these are also ones with no person_id, proper dates, or derived tables  

#Your loop references these tables
#You run this to load the list of tables into the dataframe "table"

sql3 <-paste('select distinct table_name  from ' ,sourcedb,'.INFORMATION_SCHEMA.COLUMNS where table_name like \'cb%\' order by table_name ;', sep = "")
tb3 <- bq_project_query(project_id, sql3)

table2 <- bq_table_download(tb3)
#table2

#Drop them first
for(i in 1:nrow(table2)) 
{
# for-loop over columns
    #print(paste0("Table: ", table2[[i, 1]]))
    temp_sql_query  <-paste('drop table if exists ',targetdb,'.', table2[[i, 1]] ,';' , sep = "") 
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(temp_sql_query)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}

eric <- paste("cb tables " ,  targetdb ," dropped " ,Sys.time() + hours(1) ,sep = "")
print (eric)

print('Done dropping cb tables')

#Now build them 
for(i in 1:nrow(table2)) 
{
# for-loop over columns
    #print(paste0("Table: ", table2[[i, 1]]))
    temp_sql_query  <-paste('create table ',targetdb,'.',table2[[i, 1]] ,' as select src.* from ' ,sourcedb,'.',table2[[i, 1]],' src', sep = "")
    #print(temp_sql_query)
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}

fred <-paste('Done building cb tables  ' ,Sys.time() + hours(1), sep = "")
print (fred)

#Section to build user tables with no person_id or derived tables  
#These are specifically referenced


ERROR: [1m[33mError[39m in `vectbl_as_row_location2()`:[22m
[33m![39m Can't extract rows past the end.
[1m[22m[36mℹ[39m Location 1 doesn't exist.
[36mℹ[39m There are only 0 rows.


In [23]:
sql90 <-paste('drop table if exists ',targetdb,'.tmp_persons;', sep = "")
tb90 <-bq_project_query(project_id, sql90)

sql91 <-paste('drop table if exists ',targetdb,'.tmp_Eventdates;', sep = "")
tb91 <-bq_project_query(project_id, sql91)

sql92 <-paste('drop table if exists ',targetdb,'.tmp_EventdatesValid;', sep = "")
tb92 <-bq_project_query(project_id, sql92)

print("temp tables dropped")

[1] "temp tables dropped"


In [24]:

fred <-paste('FDM Build of',targetdb,'finished')
print (fred)

[1] "FDM Build of yhcr-prd-phm-bia-core.CB_FDM_YAS_V1 finished"
